In [45]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import re
from num2words import num2words
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [4]:
corpus = ['Apple Orange Orange Apple','Apple Banana Apple Banana','Banana Apple Banana Banana Banana Apple',\
          'Banana Orange Banana Banana Orange Banana','Banana Apple Banana Banana Orange Banana']

In [5]:
''' Create a function to vectorize the counts of a list of sentences. Does not ignore case.'''
def myCountVec(corpus):
    bag = set()  # use a set to figure out how many unique words there are
    word_index = dict() # use a dict to store the column index of the word
    for line in corpus:
        for word in line.split():
            if word in bag:
                continue
            else:
                bag.add(word)
    list_of_words = sorted(list(bag)) # sort the list of words derived from set
    for index,word in enumerate(list_of_words):
        word_index[word] = index  # assign index to word
    vec_of_counts = np.zeros((len(corpus),len(word_index))) # create dummy matrix with all locations initilised to 0
    for row,line in enumerate(corpus):
        for word in line.split():
            col = word_index[word]
            vec_of_counts[row][col] += 1  # populate the matrix, increasing a word count when needed.
    return vec_of_counts

In [6]:
myCountVec(corpus)

array([[2., 0., 2.],
       [2., 2., 0.],
       [2., 4., 0.],
       [0., 4., 2.],
       [1., 4., 1.]])

In [7]:
df = pd.read_csv('MrTrumpSpeeches.csv', sep='~', encoding='latin1')
df['sentiment'] = np.where(df['like_count'] > df['dislike_count'], 1, 0)
df.head()

,id,playlist,upload_date,title,view_count,average_rating,like_count,dislike_count,subtitles,sentiment
0,-2WTNSujhjk,Donald Trump Speeches & Events,20160220,Live Stream: Donald Trump Victory Rally in Spa...,4057.0,4.259259,44.0,10.0,presidents of the United States mr. go tr...,1
1,-64nfy6i58w,Donald Trump Speeches & Events,20161107,LAST RALLY: Donald Trump FINAL CAMPAIGN Rally ...,47276.0,4.358025,952.0,182.0,it's now officially Tuesday November a di...,1
2,-7Sp31hTxkU,Donald Trump Speeches & Events,20160423,"FULL SPEECH: Donald Trump Rally in Bridgeport,...",19966.0,4.666667,220.0,20.0,you [Music] [Music] [Music] you I...,1
3,-byuyavcNI4,Donald Trump Speeches & Events,20160617,"Full Speech: Donald Trump Rally in Houston, Te...",15138.0,4.582491,266.0,31.0,we welcome stars and president [Music] ...,1
4,09BXh-AA72M,Donald Trump Speeches & Events,20161105,"Full Speech: Donald Trump Rally in Denver, Col...",8720.0,4.924731,365.0,7.0,you thank you [Music] great people Gr...,1


In [9]:
# before we clean, lets have a look at a word dictionary to see if we can spot some odd things
D = {}
for line in df['subtitles']:
    for word in line.split():
        if word.lower() in D:
            D[word.lower()] += 1
        else:
            D[word.lower()] = 1
D1 = OrderedDict(sorted(D.items()))
D1.items()

odict_items([('$1', 2), ('$1,000', 1), ('$1,200', 1), ('$1,500', 6), ('$10', 1), ('$100,000', 9), ('$10000', 1), ('$100000', 10), ('$12,000', 1), ('$12000', 1), ('$15,000', 3), ('$1500', 6), ('$15000', 2), ('$18,000', 1), ('$2,000', 2), ('$24000', 1), ('$25', 1), ('$25,000', 1), ('$2500', 1), ('$25000', 1), ('$30000', 1), ('$3700', 1), ('$400,000', 1), ('$5,000', 2), ('$50,000', 4), ('$500', 2), ('$500,000', 1), ('$5000', 1), ('$50000', 4), ('$53', 1), ('$6300', 1), ('$64', 1), ('$69', 1), ('$7', 1), ('$7000', 2), ('$71', 2), ('$8', 1), ('%', 8), ('&amp;', 18), ("'", 1), ("'add", 1), ("'as", 1), ("'he's", 1), ("'i", 3), ("'s", 1), ('*', 1), ('+', 1), (',', 1), ('-', 62), ('--', 15), ('--nt', 1), ('--nz', 1), ('/', 42), ('0', 15), ('0%', 3), ('00', 6), ('000', 3), ('01', 5), ('06', 2), ('07', 1), ('08', 1), ('1', 154), ('1%', 7), ('1,000', 11), ('1,000,000', 1), ('1,100', 4), ('1,200', 1), ('1,300', 1), ('1,400', 13), ('1,500', 1), ('1,800', 3), ('1-yard', 1), ('1.1', 1), ('1.2', 10), (

In [43]:
# so there are some genral punctuation marks and brackets that should be removed since they dont add anything to the context.
# there also are quite a few words that are numbers. We will try to change these to text using num2words
def cleaningFunc(line):
    line = re.sub('[\/\+\-,:.\'\$*%\[\]\&]','',line)
    words = line.split()
    newwords = []
    for word in words:
        if word.isdigit():
            newwords.append(re.sub('[ ,-]','',num2words(int(word))))
        else:
            newwords.append(word)
    return " ".join(newwords)

In [44]:
df['subtitle_clean'] = df['subtitles'].apply(lambda x : cleaningFunc(x))
D = {}
for line in df['subtitle_clean']:
    for word in line.split():
        if word.lower() in D:
            D[word.lower()] += 1
        else:
            D[word.lower()] = 1
D1 = OrderedDict(sorted(D.items()))
D1.items()

odict_items([('100day', 15), ('100percent', 1), ('100th', 3), ('105th', 1), ('10step', 6), ('10th', 6), ('10year', 5), ('117th', 1), ('11th', 22), ('13th', 2), ('140character', 1), ('14th', 2), ('14under', 3), ('150th', 8), ('150v', 1), ('15minute', 1), ('15percent', 1), ('15th', 5), ('16th', 20), ('16wheeler', 1), ('17point', 2), ('17th', 6), ('1880s', 3), ('18th', 3), ('18wheeler', 3), ('1920s', 2), ('1940s', 6), ('1950s', 4), ('1960s', 3), ('196th', 1), ('1970s', 6), ('1980s', 6), ('1990s', 12), ('1991s', 2), ('19th', 1), ('1i', 2), ('1pm', 1), ('1st', 16), ('1yard', 1), ('200day', 1), ('200year', 5), ('20minute', 2), ('20point', 4), ('20s', 10), ('20th', 44), ('21gun', 1), ('21percent', 1), ('21point', 2), ('21st', 15), ('22nd', 1), ('22year', 1), ('23rd', 2), ('24day', 1), ('24hour', 2), ('24th', 5), ('250th', 2), ('25cent', 1), ('25percent', 2), ('25point', 3), ('25th', 7), ('25year', 2), ('26nominations', 3), ('26th', 1), ('27th', 3), ('27year', 1), ('28member', 2), ('28th', 2),

In [38]:
# ok so thats our dictioanry of words. Lets pass it into CountVectorizer and TDIDFVEctorizer


1